# Price prediction using LSTM

Using an LSTM to predict price changes in the value of Bitcoin and gold.  Then, using predicted values to decide what changes to make to our portfolio at each day.

## Bitcoin price prediction model

Inspired by *https://www.kaggle.com/code/meetnagadia/bitcoin-price-prediction-using-lstm/notebook*

### Import libraries

In [3]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# Evaluation libraries
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Model libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM


# Plotting libraries
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### Load dataset

In [22]:
bc_df = pd.read_csv("BCHAIN-MKPRU.csv")
print(f"Total number of days: {bc_df.shape[0]}")
print(f"Number of columns: {bc_df.shape[1]}")
print(f"Null values: {bc_df.isnull().values.sum()}")

# Make sure date column has datetime objects
bc_df['Date'] = pd.to_datetime(bc_df['Date'])
bc_df.head()

Total number of days: 1826
Number of columns: 2
Null values: 0


C:\Users\n8moo\AppData\Local\Temp\ipykernel_12576\3529312939.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bc_df['Date'] = pd.to_datetime(bc_df['Date'])


,Date,Value
0,2016-09-11,621.65
1,2016-09-12,609.67
2,2016-09-13,610.92
3,2016-09-14,608.82
4,2016-09-15,610.38


In [23]:
# Start and end dates
start_date = bc_df.iloc[0]['Date']
end_date = bc_df.iloc[-1]['Date']
print(start_date, end_date)

2016-09-11 00:00:00 2021-09-10 00:00:00


In [26]:
# Get first year of values
first_year = bc_df[bc_df['Date'] < '2017-09-11']
print(f"Total data for prediction: {first_year.shape[0]}")

# Plot first year of data
fig = px.line(first_year, x=first_year.Date, y=first_year.Value, labels={'date':'Date','value':'Value'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered period to predict Bitcoin close price', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Total data for prediction: 365


#### Normalize data

In [29]:
# Normalize data

# Delete date column because it can't be normalized
del bc_df['Date']

scaler = MinMaxScaler(feature_range=(0,1))
bc_df = scaler.fit_transform(np.array(bc_df).reshape(-1,1))
print(bc_df.shape)

(1826, 1)


#### Split into training/test data

In [ ]:
training_size = int(len(bc_df) * 0.6)  # Training data is first 60% of all data
test_size = len(bc_df) - training_size
train_data, test_data = bc_df[0:training_size, :], bc_df[training_size:len(bc_df) - 1]

print(train_data.shape, test_data.shape)

(1095, 1) (730, 1)


**TODO**: Time series analysis thing